# DENTIST R and cpp cpmparison


## Overview

DENTIST code is implemented in cpp `https://github.com/Yves-CHEN/DENTIST`. To incorporate it in the pecotmr package an R version is developed. 

Here we will test if the R version code have the same result of cpp version.

## Important note

DENTIST does not have a easy way to compile it in the system. Their paths in the makefile is quite messy and we need to install all dependencies and re-wrote the make file, which is quite time consuming. 

However, DENTIST provided an excutable file that can do the same things, but we cannot modify it so that we can only have input and outputs. Given that DENTIST involves randomness in splitting the region into S1 and S2 two parts and iterated this multiple times, so the numerical value of different attempts might be different. So here our strategy is to run DENTIST 10 times in each system, take the average for numerical things and compare if they are similar enough.

## Input

DENTIST the original program needs PLINK file as input, and also sumstat in COJO format (including columns `SNP A1 A2 freq b se p N`). Here we use the first round RSS_QC result as reference, found a region have outliers after allele QC. We use this region to verify the similarity of result.

+ PLINK file: `/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE`

+ Sumstat: `/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt`


Notes about inputs: DENTIST only allow region with 2000+ variables, also, should not include variants that have maf = 0 in PLINK. So here the plink file is already filtered by threshold 0.01.

## Details

### 1. MWE data preparation

For DENTIST in github https://github.com/Yves-CHEN/DENTIST version 1.3.0.0, it requires PLINK data, so here I prepared it.

In [6]:
library(tidyverse)
library(susieR)
library(plink2R)
library(pecotmr)
library(data.table)

# Original sumstat is from Bellenguez GWAS data. 
# I found a region that has some signal and outliers so perfect for MWE.
# Keep ~ 4000 variants because it's demanded by DENTIST

test_data_path = "/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Original_sumstat.tsv"
snp_data <- fread(test_data_path)

# get variant list to extract from PLINK data
snp_data %>% mutate(variant_id = paste0("chr", chrom, ":", pos, "_", A2, "_", A1)) %>% pull(variant_id) %>% 
    write.table("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_snplist.txt", row.names = F, quote = F)


Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [ ]:
plink --bfile /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/geno_by_chrom/ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.1 \ 
    --extract /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_snplist.txt \
    --make-bed --maf 0.01 --geno 0.01 --out ~/RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE

In [21]:
genotype = read_plink("~/RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE")

# filter out columns that contain NA so that LD matrix can be computed without error
keep_index = which(!is.na(colSums(genotype$bed)))

names(keep_index) %>% 
    write.table("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_snplist.txt", row.names = F, quote = F)

snp_data %>% mutate(variant_id = paste0("chr", chrom, ":", pos, "_", A2, "_", A1)) %>%
    filter(variant_id %in% names(keep_index)) %>% mutate(pvalue = pnorm(beta / se)) %>%
    select(variant_id, A1, A2, maf, beta, se,  pvalue ,n_case) %>% rename(SNP = variant_id,
                                                                        freq = maf, b = beta,
                                                                        p = pvalue, N = n_case) %>%
    mutate(N = 1153) %>%
    write_tsv("~/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt")

In [ ]:
# rerun plink to remove those variants containing NA
plink --bfile /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/geno_by_chrom/ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.1 \ 
    --extract /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_snplist.txt \
    --make-bed --maf 0.01 --geno 0.01 --out ~/RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE

In [ ]:
head -n 6 /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_snplist.txt >> /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_top5_snplist.txt
plink --bfile /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/geno_by_chrom/ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.1 --extract /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/MWE_top5_snplist.txt --make-bed --maf 0.01 --geno 0.01 --out /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_top5_MWE

In [39]:
# calculate LD matrix to run it in rCPP interface 
#sumstat = read_delim("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt")
#geno = read_plink("/home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE")$bed
#LD = as.matrix(cor(geno))
#write.table(LD, "/home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv", sep = "\t", quote = FALSE, row.names = TRUE, col.names = TRUE)

#########FINISHED############
sumstat = read_delim("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt")
LD = fread("/home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv")[,-1]
LD = as.matrix(LD)

Rows: 4123 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): SNP, A1, A2
dbl (5): freq, b, se, p, N

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 4123 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): SNP, A1, A2
dbl (5): freq, b, se, p, N

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in fread("/home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv"):
“Detected 4123 column names but the data has 4124 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that 

### 2. DENTIST  -- github compiled version implementation

In [ ]:
# DENTIST -- github compiled version

~/RSS_QC/DENTIST/DENTIST_1.3.0.0  --bfile  /home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE --gwas-summary /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt --out DENTIST_result

In [20]:
wc -l ~/RSS_QC/DENTIST/DENTIST_result.DENTIST.short.txt

63 /home/hs3393/RSS_QC/DENTIST/DENTIST_result.DENTIST.short.txt


Here the row number of *.short.txt shows how many outliers are identified in this region and should be removed.

So for DENTIST compiled version, the **number of outlier is 63**. 

### 3. DENTIST -- Rcpp version

In [35]:
sumstat = read_delim("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt") %>% mutate(z = b / se)
LD = fread("/home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv")[,-1]
LD = as.matrix(LD)
dentist_result = dentist(zScore = sumstat$z, LD = LD, nSample = 1153)

Rows: 4123 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): SNP, A1, A2
dbl (5): freq, b, se, p, N

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in fread("/home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv"):
“Detected 4123 column names but the data has 4124 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [36]:
dentist_result %>% filter(is_problematic > 0) %>% nrow()

[1] 1053

So for DENTIST RCPP version, the **number of outlier is 1053**. The results looks quite different... Take a look at the LD.

In [52]:
library(readr)
library(dplyr, warn.conflicts = FALSE)
library(data.table)
library(RcppArmadillo)
library(Rcpp)
source("/home/rd2972/software/pecotmr/R/run_dentist.R")
source("/home/rd2972/software/pecotmr/R/RcppExports.R")
sourceCpp("/home/hs3393/RSS_QC/pecotmr/src/dentist.cpp")
sourceCpp("/home/rd2972/software/pecotmr/src/RcppExports.cpp")
#dentist_LD_result = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/output_LD.txt")

Warning message in sourceCpp("/home/rd2972/software/pecotmr/src/RcppExports.cpp"):
“No Rcpp::export attributes or RCPP_MODULE declarations found in source”


In [55]:
LD

chr1:206011748_A_G,chr1:206012565_A_G,chr1:206012721_C_G,chr1:206013858_A_G,chr1:206014205_G_A,chr1:206015713_G_A,chr1:206016996_A_T,chr1:206018106_T_C,chr1:206018145_C_T,chr1:206018463_T_C,⋯,chr1:208455523_C_T,chr1:208455658_G_A,chr1:208455862_A_G,chr1:208456632_T_C,chr1:208457317_A_C,chr1:208457639_C_T,chr1:208458441_G_A,chr1:208459950_T_C,chr1:208460159_A_G,chr1:208461262_G_A
1.00000000,0.98500629,0.98215134,0.97722319,-0.16241358,0.98013632,0.94805046,0.94283218,0.312660171,0.41408706,⋯,-0.021963764,-0.0193532592,-0.0138176149,-0.039436222,0.0256967804,-0.005392992,-0.0135394991,-0.0461785952,-0.044373583,-0.0411658907
0.98500629,1.00000000,0.99703144,0.98655338,-0.16597212,0.98062367,0.94822880,0.95174415,0.305991357,0.40531837,⋯,-0.024245263,-0.0216382623,-0.0055390197,-0.041944429,0.0225030342,-0.007702361,-0.0153967323,-0.0429995460,-0.041161901,-0.0382699506
0.98215134,0.99703144,1.00000000,0.98367739,-0.16673209,0.97479826,0.94244220,0.94886559,0.304141005,0.40288333,⋯,-0.024789252,-0.0221849130,-0.0042933214,-0.042531913,0.0217025520,-0.008264625,-0.0158425589,-0.0437020510,-0.041865722,-0.0390144232
0.97722319,0.98655338,0.98367739,1.00000000,-0.16553057,0.99096828,0.95579392,0.96236864,0.309334231,0.40969015,⋯,-0.022075062,-0.0194857825,-0.0114711522,-0.039436839,0.0250915751,-0.005639725,-0.0136646556,-0.0400754211,-0.038252242,-0.0352470225
-0.16241358,-0.16597212,-0.16673209,-0.16553057,1.00000000,-0.16225296,-0.07833410,-0.08209722,-0.033804260,-0.05615093,⋯,-0.021080030,0.0368082378,0.0606923459,-0.019753167,-0.0102328423,-0.021702814,-0.0279977193,-0.0072278676,-0.005818709,0.0187061610
0.98013632,0.98062367,0.97479826,0.99096828,-0.16225296,1.00000000,0.96493215,0.95671660,0.313204436,0.41476782,⋯,-0.020266689,-0.0262699565,-0.0152635169,-0.037391336,0.0274109927,-0.003872517,-0.0122088692,-0.0376714618,-0.035855042,-0.0327423041
0.94805046,0.94822880,0.94244220,0.95579392,-0.07833410,0.96493215,1.00000000,0.99127458,0.299981014,0.40379655,⋯,-0.027551593,0.0003804079,-0.0003697598,-0.045727553,0.0330202418,-0.010885808,-0.0076633060,-0.0474268837,-0.045571677,-0.0369573921
0.94283218,0.95174415,0.94886559,0.96236864,-0.08209722,0.95671660,0.99127458,1.00000000,0.296396524,0.39906019,⋯,-0.029298390,-0.0014986479,0.0032685364,-0.047709651,0.0307219444,-0.012585963,-0.0091252968,-0.0497536193,-0.047891132,-0.0394115749
0.31266017,0.30599136,0.30414100,0.30933423,-0.03380426,0.31320444,0.29998101,0.29639652,1.000000000,-0.03499675,⋯,-0.016766301,-0.0160300508,0.0370233024,-0.008195615,-0.0242489625,0.038052609,-0.0026999291,-0.0209378224,-0.020386566,-0.0065450325
0.41408706,0.40531837,0.40288333,0.40969015,-0.05615093,0.41476782,0.40379655,0.39906019,-0.034996747,1.00000000,⋯,-0.034190273,-0.0151785978,0.0181593439,-0.046386610,-0.0266248820,0.009562230,-0.0189365839,-0.0321306984,-0.031440272,-0.0603098837


In [54]:
result1 = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/20240308_version/DENTIST_Rcpp_output_01.txt")

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V2052,V2053,V2054,V2055,V2056,V2057,V2058,V2059,V2060,V2061
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.985010,0.997030,-0.165970,0.980620,0.9517400,0.3059900,0.1963500,0.24287000,1.9603e-01,0.200050,⋯,0.0159500,0.01011600,-0.00949390,-1.4634e-03,-0.0104610,-0.0104610,-0.0242450,-0.00553900,-0.0419440,-0.0430000
0.977220,0.983680,-0.165530,0.990970,0.9623700,0.3093300,0.1821200,0.24568000,1.9891e-01,0.184900,⋯,0.0252130,0.01956100,-0.01308400,4.3563e-03,-0.0073216,-0.0073216,-0.0220750,-0.01147100,-0.0394370,-0.0400750
0.948050,0.942440,-0.078334,0.964930,0.9912700,0.2999800,0.1746400,0.23784000,1.9103e-01,0.177670,⋯,0.0142890,0.00809080,-0.01694700,9.1581e-05,-0.0035575,-0.0035575,-0.0275520,-0.00036976,-0.0457280,-0.0474270
0.414090,0.402880,-0.056151,0.414770,0.3990600,-0.0349970,-0.0490780,-0.02528900,-3.2538e-02,-0.046423,⋯,0.0057649,-0.00058517,-0.04052900,-1.2314e-03,-0.0481700,-0.0481700,-0.0341900,0.01815900,-0.0463870,-0.0321310
0.582510,0.574340,-0.182270,0.589240,0.6041700,0.1733100,0.0681870,0.16826000,8.4623e-02,0.077967,⋯,-0.0131740,-0.02404500,0.02470500,-6.2777e-03,0.0038312,0.0038312,-0.0474740,0.04000800,-0.0347840,-0.0316340
0.289360,0.280880,-0.048627,0.290170,0.2728200,0.9004400,0.7084100,-0.03901300,-7.5352e-05,0.724340,⋯,-0.0075276,-0.01285700,-0.02182200,-3.2347e-03,-0.0328830,-0.0328830,-0.0233400,0.04111700,-0.0169860,-0.0147250
0.197800,0.210030,-0.066790,0.198570,0.1854400,0.6333200,0.9384300,-0.00181520,1.8330e-02,0.987060,⋯,-0.0296610,-0.03375500,-0.01208400,1.7388e-03,-0.0503800,-0.0503800,-0.0097299,0.06845400,-0.0203450,-0.0299390
0.183230,0.213230,-0.064954,0.184010,0.1714600,0.5908600,0.9381800,-0.00176530,1.7826e-02,0.986700,⋯,-0.0288450,-0.03282700,-0.01175200,8.5082e-03,-0.0489950,-0.0489950,-0.0094623,0.06657200,-0.0197860,-0.0291160
0.186980,0.216420,-0.066252,0.187760,0.1750000,0.6072600,0.9508200,-0.00257810,1.6673e-02,1.000000,⋯,-0.0333890,-0.03739900,-0.01365600,8.8226e-03,-0.0497280,-0.0497280,-0.0102980,0.07230000,-0.0206620,-0.0300530


ERROR: Error in parse(text = x, srcfile = src): <text>:1:150: unexpected input
1: result2 = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/20240308_version/DENTIST_Rcpp_output_11.txt"）
                                                                                                                                                         ^


In [68]:
fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/20240308_version/DENTIST_Rcpp_output_11.txt")

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V1837,V1838,V1839,V1840,V1841,V1842,V1843,V1844,V1845,V1846
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.0416310,-0.0414220,0.01065200,-0.0392200,-0.0086544,0.0083688,-0.0268740,-0.0174780,0.0345340,-0.00435830,⋯,-0.02989400,0.0119690,-0.03558100,0.1079800,0.0500620,-0.0219700,0.1420100,-0.00353260,-0.0099891,-0.0270660
-0.0271160,-0.0334570,0.01177400,-0.0349700,-0.0307070,0.0151990,-0.0239620,-0.0475550,-0.0274190,-0.02453400,⋯,-0.02665500,-0.0059261,0.05736400,0.0012365,-0.0040265,-0.1007200,-0.0371600,0.03521400,-0.0525440,-0.0241330
-0.0150950,0.0788840,0.01142900,0.0272150,0.0175620,-0.0107720,-0.0132490,-0.0045647,-0.0306140,0.03286100,⋯,0.12661000,0.0257590,0.04525800,-0.0337110,0.0714320,-0.0764010,-0.0162190,-0.00033008,-0.0040773,-0.0344950
-0.0747820,0.0207080,-0.00889540,-0.0468830,-0.0358610,-0.0107720,0.0631760,0.4878400,-0.0067717,-0.03132500,⋯,-0.02180100,0.0535490,-0.03340300,-0.0450920,0.0092160,0.2327200,0.0069310,-0.06035200,0.9954800,0.0167560
-0.0360670,-0.0478700,-0.20013000,0.0109740,0.0022737,0.0112420,0.0196730,0.0184430,0.0300300,0.08387500,⋯,0.04241600,-0.0078417,-0.03917600,0.0294250,-0.0162560,0.0309220,0.0374260,-0.11064000,0.0605240,-0.0705380
0.0703750,-0.0192480,-0.32660000,-0.0708390,-0.0766960,0.0011983,-0.0224980,0.0407870,-0.0428860,-0.00590880,⋯,-0.07980000,0.1622600,0.08114500,-0.1521500,0.0339110,0.0574500,-0.0252190,0.11850000,0.0668080,0.0290330
-0.0535730,0.1145300,0.02604700,-0.0591690,-0.0409050,0.0435680,-0.0259500,0.0447860,0.0090358,0.00513640,⋯,-0.01608200,-0.0334290,0.02822500,-0.0626970,-0.0236540,-0.0874280,-0.0461230,-0.00557530,0.0544290,-0.0181160
0.0502220,-0.0088200,0.03224900,0.0054500,0.0342550,-0.0157530,0.0277480,0.0043595,0.0092411,0.01186800,⋯,-0.02353600,0.0100630,0.15454000,-0.0052423,0.2763800,-0.0260020,-0.0074602,0.04702600,0.0101720,0.0426810
-0.0266670,0.1907900,0.00052716,-0.0060007,0.0028529,-0.0313640,-0.0030392,-0.0177610,-0.0075308,0.00662770,⋯,0.06910100,0.0184730,-0.00208010,0.0098927,-0.0220870,-0.0571130,-0.0330030,-0.03836000,-0.0285440,0.0221770


In [58]:
result2 

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V2052,V2053,V2054,V2055,V2056,V2057,V2058,V2059,V2060,V2061
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.985010,0.997030,-0.165970,0.980620,0.9517400,0.3059900,0.1963500,0.24287000,1.9603e-01,0.200050,⋯,0.0159500,0.01011600,-0.00949390,-1.4634e-03,-0.0104610,-0.0104610,-0.0242450,-0.00553900,-0.0419440,-0.0430000
0.977220,0.983680,-0.165530,0.990970,0.9623700,0.3093300,0.1821200,0.24568000,1.9891e-01,0.184900,⋯,0.0252130,0.01956100,-0.01308400,4.3563e-03,-0.0073216,-0.0073216,-0.0220750,-0.01147100,-0.0394370,-0.0400750
0.948050,0.942440,-0.078334,0.964930,0.9912700,0.2999800,0.1746400,0.23784000,1.9103e-01,0.177670,⋯,0.0142890,0.00809080,-0.01694700,9.1581e-05,-0.0035575,-0.0035575,-0.0275520,-0.00036976,-0.0457280,-0.0474270
0.414090,0.402880,-0.056151,0.414770,0.3990600,-0.0349970,-0.0490780,-0.02528900,-3.2538e-02,-0.046423,⋯,0.0057649,-0.00058517,-0.04052900,-1.2314e-03,-0.0481700,-0.0481700,-0.0341900,0.01815900,-0.0463870,-0.0321310
0.582510,0.574340,-0.182270,0.589240,0.6041700,0.1733100,0.0681870,0.16826000,8.4623e-02,0.077967,⋯,-0.0131740,-0.02404500,0.02470500,-6.2777e-03,0.0038312,0.0038312,-0.0474740,0.04000800,-0.0347840,-0.0316340
0.289360,0.280880,-0.048627,0.290170,0.2728200,0.9004400,0.7084100,-0.03901300,-7.5352e-05,0.724340,⋯,-0.0075276,-0.01285700,-0.02182200,-3.2347e-03,-0.0328830,-0.0328830,-0.0233400,0.04111700,-0.0169860,-0.0147250
0.197800,0.210030,-0.066790,0.198570,0.1854400,0.6333200,0.9384300,-0.00181520,1.8330e-02,0.987060,⋯,-0.0296610,-0.03375500,-0.01208400,1.7388e-03,-0.0503800,-0.0503800,-0.0097299,0.06845400,-0.0203450,-0.0299390
0.183230,0.213230,-0.064954,0.184010,0.1714600,0.5908600,0.9381800,-0.00176530,1.7826e-02,0.986700,⋯,-0.0288450,-0.03282700,-0.01175200,8.5082e-03,-0.0489950,-0.0489950,-0.0094623,0.06657200,-0.0197860,-0.0291160
0.186980,0.216420,-0.066252,0.187760,0.1750000,0.6072600,0.9508200,-0.00257810,1.6673e-02,1.000000,⋯,-0.0333890,-0.03739900,-0.01365600,8.8226e-03,-0.0497280,-0.0497280,-0.0102980,0.07230000,-0.0206620,-0.0300530


In [56]:
result1 = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/20240308_version/DENTIST_Rcpp_output_02.txt")

In [53]:
dentist_result = dentist(zScore = sumstat$z, LD = LD, nSample = 1153)

ERROR: Error in value[[3L]](cond): Adjusted rsq_eigen value exceeding 1: 1.002794


In [52]:
dim(dentist_LD_result)
head(dentist_LD_result)

[1] 2062 2061

LD_it,for,iteration,:,V5,V6,V7,V8,V9,V10,⋯,V2052,V2053,V2054,V2055,V2056,V2057,V2058,V2059,V2060,V2061
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.98501,0.99703,-0.165970,0.98062,0.95174,0.305990,0.196350,0.242870,1.9603e-01,0.200050,⋯,0.0159500,0.01011600,-0.0094939,-1.4634e-03,-0.0104610,-0.0104610,-0.024245,-0.00553900,-0.041944,-0.043000
0.97722,0.98368,-0.165530,0.99097,0.96237,0.309330,0.182120,0.245680,1.9891e-01,0.184900,⋯,0.0252130,0.01956100,-0.0130840,4.3563e-03,-0.0073216,-0.0073216,-0.022075,-0.01147100,-0.039437,-0.040075
0.94805,0.94244,-0.078334,0.96493,0.99127,0.299980,0.174640,0.237840,1.9103e-01,0.177670,⋯,0.0142890,0.00809080,-0.0169470,9.1581e-05,-0.0035575,-0.0035575,-0.027552,-0.00036976,-0.045728,-0.047427
0.41409,0.40288,-0.056151,0.41477,0.39906,-0.034997,-0.049078,-0.025289,-3.2538e-02,-0.046423,⋯,0.0057649,-0.00058517,-0.0405290,-1.2314e-03,-0.0481700,-0.0481700,-0.034190,0.01815900,-0.046387,-0.032131
0.58251,0.57434,-0.182270,0.58924,0.60417,0.173310,0.068187,0.168260,8.4623e-02,0.077967,⋯,-0.0131740,-0.02404500,0.0247050,-6.2777e-03,0.0038312,0.0038312,-0.047474,0.04000800,-0.034784,-0.031634
0.28936,0.28088,-0.048627,0.29017,0.27282,0.900440,0.708410,-0.039013,-7.5352e-05,0.724340,⋯,-0.0075276,-0.01285700,-0.0218220,-3.2347e-03,-0.0328830,-0.0328830,-0.023340,0.04111700,-0.016986,-0.014725


Ignore the column names. The dimension of LD is not correct yet, also the diagnol elements are not equal to 1.

In [58]:
DENTIST_compile = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/DENTIST_Rcpp_output.txt")

Warning message in fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/DENTIST_Rcpp_output.txt"):
“Detected 4 column names but the data has 2061 columns (i.e. invalid file). Added 2057 extra default column names at the end.”
Warning message in fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/DENTIST_Rcpp_output.txt"):
“Stopped early on line 2064. Expected 2061 fields but found 0. Consider fill=TRUE and comment.char=. First discarded non-empty line: <<LD_it for iteration :>>”


In [59]:
DENTIST_compile

LD_it,for,iteration,:,V5,V6,V7,V8,V9,V10,⋯,V2052,V2053,V2054,V2055,V2056,V2057,V2058,V2059,V2060,V2061
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.985010,0.997030,-0.165970,0.980620,0.9517400,0.3059900,0.1963500,0.24287000,1.9603e-01,0.200050,⋯,0.0159500,0.01011600,-0.00949390,-1.4634e-03,-0.0104610,-0.0104610,-0.0242450,-0.00553900,-0.0419440,-0.0430000
0.977220,0.983680,-0.165530,0.990970,0.9623700,0.3093300,0.1821200,0.24568000,1.9891e-01,0.184900,⋯,0.0252130,0.01956100,-0.01308400,4.3563e-03,-0.0073216,-0.0073216,-0.0220750,-0.01147100,-0.0394370,-0.0400750
0.948050,0.942440,-0.078334,0.964930,0.9912700,0.2999800,0.1746400,0.23784000,1.9103e-01,0.177670,⋯,0.0142890,0.00809080,-0.01694700,9.1581e-05,-0.0035575,-0.0035575,-0.0275520,-0.00036976,-0.0457280,-0.0474270
0.414090,0.402880,-0.056151,0.414770,0.3990600,-0.0349970,-0.0490780,-0.02528900,-3.2538e-02,-0.046423,⋯,0.0057649,-0.00058517,-0.04052900,-1.2314e-03,-0.0481700,-0.0481700,-0.0341900,0.01815900,-0.0463870,-0.0321310
0.582510,0.574340,-0.182270,0.589240,0.6041700,0.1733100,0.0681870,0.16826000,8.4623e-02,0.077967,⋯,-0.0131740,-0.02404500,0.02470500,-6.2777e-03,0.0038312,0.0038312,-0.0474740,0.04000800,-0.0347840,-0.0316340
0.289360,0.280880,-0.048627,0.290170,0.2728200,0.9004400,0.7084100,-0.03901300,-7.5352e-05,0.724340,⋯,-0.0075276,-0.01285700,-0.02182200,-3.2347e-03,-0.0328830,-0.0328830,-0.0233400,0.04111700,-0.0169860,-0.0147250
0.197800,0.210030,-0.066790,0.198570,0.1854400,0.6333200,0.9384300,-0.00181520,1.8330e-02,0.987060,⋯,-0.0296610,-0.03375500,-0.01208400,1.7388e-03,-0.0503800,-0.0503800,-0.0097299,0.06845400,-0.0203450,-0.0299390
0.183230,0.213230,-0.064954,0.184010,0.1714600,0.5908600,0.9381800,-0.00176530,1.7826e-02,0.986700,⋯,-0.0288450,-0.03282700,-0.01175200,8.5082e-03,-0.0489950,-0.0489950,-0.0094623,0.06657200,-0.0197860,-0.0291160
0.186980,0.216420,-0.066252,0.187760,0.1750000,0.6072600,0.9508200,-0.00257810,1.6673e-02,1.000000,⋯,-0.0333890,-0.03739900,-0.01365600,8.8226e-03,-0.0497280,-0.0497280,-0.0102980,0.07230000,-0.0206620,-0.0300530


In [28]:
kk = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/LD_separate/LD_it_output_20.txt")

In [29]:
head(kk)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V973,V974,V975,V976,V977,V978,V979,V980,V981,V982
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.6759160,-0.05131040,-1.92395e-03,-0.04470260,0.0505993,-0.0797282,0.0436569,0.01908220,0.32164800,0.05074640,⋯,0.1378230,0.11253900,0.03901070,-0.01396100,-0.0256802,-0.08519900,0.1151590,-0.2119800,0.04571330,-0.1430800
-0.0286956,0.02563550,1.87835e-01,-0.03293410,0.0358906,0.0264595,-0.0358579,-0.04175240,-0.03144050,-0.03414390,⋯,0.0164742,0.00802881,0.00357216,0.02387510,0.0708860,-0.03225810,-0.0545845,0.0235798,-0.00985897,-0.0141408
-0.0742423,0.18604700,7.14334e-02,0.03134780,-0.0447789,0.1843570,-0.1723980,-0.02332210,-0.04275870,0.00418691,⋯,-0.0185382,-0.00656875,0.01116950,-0.15475400,0.0525748,0.01008800,-0.0784397,0.0335985,-0.07618340,0.0346303
-0.1909020,0.00375349,-1.68635e-03,-0.06660320,0.0394499,-0.0277484,0.0172595,-0.00700894,-0.34048000,-0.05132800,⋯,-0.0600390,-0.06478690,-0.05054220,-0.00102061,-0.0511662,0.09739920,-0.0980070,-0.0827689,-0.04171430,0.1791850
0.8078360,-0.04763070,-7.06678e-05,-0.00322825,0.0854091,-0.0509672,0.0533767,0.01319020,0.22029300,0.07152070,⋯,0.0398345,0.00410843,0.04491120,0.01159600,-0.0482848,-0.09297800,0.0922414,-0.2087910,0.06020950,-0.1235290
-0.0531262,0.00703471,2.20129e-02,0.22299300,-0.0183365,0.0137763,0.0501553,0.02820910,0.00702352,0.07164150,⋯,-0.0555523,0.04412450,0.09947470,-0.04475670,0.1023430,0.00352642,-0.0719310,0.0556175,-0.02644690,-0.0668466


In [26]:
k2 = fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/4213_variants/DENTIST_Rcpp_output_01.txt")

In [27]:
k2

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V2052,V2053,V2054,V2055,V2056,V2057,V2058,V2059,V2060,V2061
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.985010,0.997030,-0.165970,0.980620,0.9517400,0.3059900,0.1963500,0.24287000,1.9603e-01,0.200050,⋯,0.0159500,0.01011600,-0.00949390,-1.4634e-03,-0.0104610,-0.0104610,-0.0242450,-0.00553900,-0.0419440,-0.0430000
0.977220,0.983680,-0.165530,0.990970,0.9623700,0.3093300,0.1821200,0.24568000,1.9891e-01,0.184900,⋯,0.0252130,0.01956100,-0.01308400,4.3563e-03,-0.0073216,-0.0073216,-0.0220750,-0.01147100,-0.0394370,-0.0400750
0.948050,0.942440,-0.078334,0.964930,0.9912700,0.2999800,0.1746400,0.23784000,1.9103e-01,0.177670,⋯,0.0142890,0.00809080,-0.01694700,9.1581e-05,-0.0035575,-0.0035575,-0.0275520,-0.00036976,-0.0457280,-0.0474270
0.414090,0.402880,-0.056151,0.414770,0.3990600,-0.0349970,-0.0490780,-0.02528900,-3.2538e-02,-0.046423,⋯,0.0057649,-0.00058517,-0.04052900,-1.2314e-03,-0.0481700,-0.0481700,-0.0341900,0.01815900,-0.0463870,-0.0321310
0.582510,0.574340,-0.182270,0.589240,0.6041700,0.1733100,0.0681870,0.16826000,8.4623e-02,0.077967,⋯,-0.0131740,-0.02404500,0.02470500,-6.2777e-03,0.0038312,0.0038312,-0.0474740,0.04000800,-0.0347840,-0.0316340
0.289360,0.280880,-0.048627,0.290170,0.2728200,0.9004400,0.7084100,-0.03901300,-7.5352e-05,0.724340,⋯,-0.0075276,-0.01285700,-0.02182200,-3.2347e-03,-0.0328830,-0.0328830,-0.0233400,0.04111700,-0.0169860,-0.0147250
0.197800,0.210030,-0.066790,0.198570,0.1854400,0.6333200,0.9384300,-0.00181520,1.8330e-02,0.987060,⋯,-0.0296610,-0.03375500,-0.01208400,1.7388e-03,-0.0503800,-0.0503800,-0.0097299,0.06845400,-0.0203450,-0.0299390
0.183230,0.213230,-0.064954,0.184010,0.1714600,0.5908600,0.9381800,-0.00176530,1.7826e-02,0.986700,⋯,-0.0288450,-0.03282700,-0.01175200,8.5082e-03,-0.0489950,-0.0489950,-0.0094623,0.06657200,-0.0197860,-0.0291160
0.186980,0.216420,-0.066252,0.187760,0.1750000,0.6072600,0.9508200,-0.00257810,1.6673e-02,1.000000,⋯,-0.0333890,-0.03739900,-0.01365600,8.8226e-03,-0.0497280,-0.0497280,-0.0102980,0.07230000,-0.0206620,-0.0300530
